In [155]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import time

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
params={"n_estimators":500,"max_depth":6,"min_samples_split":2,"learning_rate":0.01, "loss":'ls'}
client=MongoClient()
db=client.araksa
pipeline = [{'$lookup': 
                {'from' : 'log_time',
                 'localField' : 'empId',
                 'foreignField' : 'empId',
                 'as' : 'empId'}}
             ]
#for doc in (db.emp_details.aggregate(pipeline)):
#    print(doc)
#   df1=pd.DataFrame(doc)
#    print (df1.head())


emp_log_time=db.logTime.find()
print(emp_log_time)

df_emp_log = pd.DataFrame(list(emp_log_time))
print(df_emp_log.head())
df_emp_log=df_emp_log.drop(df_emp_log.columns[0], axis=1)

df_emp_log=df_emp_log.drop('_id', axis=1)
df_emp_log=df_emp_log.drop('day', axis=1)
dict_season={'SUMMER':1, 'WINTER':2, 'AUTUM':3, 'RAINY':4}
dict_cwd={ 'AV':1,'AW':2,'ALW':3,'AH':4, 'BH':5, 'BV':6,'BW':7, 'BLW':8, 'GW':9}

df_emp_log['season'].replace(dict_season,inplace=True)
df_emp_log['cWD'].replace(dict_cwd,inplace=True)

#print(df_emp_log.head())

df_emp_log['inTime']=pd.to_numeric(df_emp_log['inTime'], errors='coerce')

df_emp_log['startTime']=pd.to_numeric(df_emp_log['startTime'], errors='coerce')

df_emp_log['distanceInKM']=pd.to_numeric(df_emp_log['distanceInKM'], errors='coerce')
df_emp_log_before=df_emp_log;
#emp_id=df_emp_log['empId']
#df_emp_log=df_emp_log.drop('empId', axis=1)

df=pd.get_dummies(df_emp_log, columns=['season','cWD'])
df=df.dropna()
print(df.head())

x=df.drop('minutesTaken',axis=1)
y=df['minutesTaken']
x_train,x_test, y_train,y_test=train_test_split(x,y, test_size=0.2, random_state=0)

gradientBoostingRegressor=GradientBoostingRegressor(**params)

gradientBoostingRegressor.fit(x_train,y_train)
r2=gradientBoostingRegressor.score(x_train,y_train)

print(r2)

r2_test=gradientBoostingRegressor.score(x_test,y_test)

print(r2_test)

y_pred=gradientBoostingRegressor.predict(x_test)


#print(x_test.head().describe())
#print(y_pred)
ts = time.time()
print(ts)
count=0
for index, row in x_test.iterrows():
    print(row['empId'])
    emp={"empId":row['empId'], "timetoTake":y_pred[count],"forDate":ts}
    print(y_pred[count])
    count=count+1
    db.predected_values.insert(emp)

#print(df_emp_log.head)






                        _class                       _id  cWD            day  \
0  com.epam.araksa.dto.LogTime  5b81d0f8fc254b3cc83b8408  NaN  1534802296784   
1  com.epam.araksa.dto.LogTime  5b81d0f8fc254b3cc83b8409  NaN  1534802296784   
2  com.epam.araksa.dto.LogTime  5b81d0f8fc254b3cc83b840a  NaN  1534802296784   
3  com.epam.araksa.dto.LogTime  5b81d0f8fc254b3cc83b840b  NaN  1534802296784   
4  com.epam.araksa.dto.LogTime  5b81d0f8fc254b3cc83b840c  NaN  1534802296784   

   distanceInKM   empId         inTime  minutesTaken  outTime season  \
0           0.0  532000  1534923960000          1100        0  RAINY   
1           0.0  534000  1534996440000          2970        0  RAINY   
2           0.0  536000  1534922940000          1620        0  RAINY   
3           0.0  538000  1534897620000           800        0  RAINY   
4           0.0  540000  1534877040000           500        0  RAINY   

       startTime  
0  1534857960000  
1  1534818240000  
2  1534825740000  
3  1534849

C:\Users\Saurabh_Mishra\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:82: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


536000
644.1107994355822
538000
462.2304188644184
536000
1894.8896733774163
532000
210.65537233834087
534000
450.8978206456862
536000
464.14999787220614
540000
676.2023868356403
540000
1524.8034818419396
540000
1883.9579871486394
534000
588.6959254037679
538000
1949.1675184382984
534000
1383.674617841757
536000
629.9072239794403
